<h1 id="basics" style="font-family:verdana;"> 
    <center>1. Setup</center>
</h1>

<div style="font-family:verdana; word-spacing:1.5px;">
    <h3 id="italic">
        1.1 loading the libraries
    </h3>
</div>

In [2]:
import pandas as pd
import numpy as np

from IPython.display import HTML

import gif
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

<div style="font-family:verdana; word-spacing:1.5px;">
    <h3 id="italic">
        1.2 Importing data
    </h3>
</div>

<div style="font-family:verdana; word-spacing:1.5px;">
    I am using the data from Kaggle: <a href="https://www.kaggle.com/jeanmidev/smart-meters-in-london" target="blank_">source</a>.
</div>

In [3]:
data = pd.read_csv('data.csv')

In [4]:
data.head()

,visibility,windBearing,temperature,time,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
0,5.97,104,10.24,2011-11-11 00:00:00,8.86,1016.76,10.24,2.77,rain,partly-cloudy-night,0.91,Partly Cloudy
1,4.88,99,9.76,2011-11-11 01:00:00,8.83,1016.63,8.24,2.95,rain,partly-cloudy-night,0.94,Partly Cloudy
2,3.70,98,9.46,2011-11-11 02:00:00,8.79,1016.36,7.76,3.17,rain,partly-cloudy-night,0.96,Partly Cloudy
3,3.12,99,9.23,2011-11-11 03:00:00,8.63,1016.28,7.44,3.25,rain,fog,0.96,Foggy
4,1.85,111,9.26,2011-11-11 04:00:00,9.21,1015.98,7.24,3.70,rain,fog,1.00,Foggy


In [5]:
print('Dataset shape: ', data.shape)

Dataset shape:  (21165, 12)


<h1 id="basics" style="font-family:verdana;"> 
    <center>2. Data Preprocessing</center>
</h1>

<div style="font-family:verdana; word-spacing:1.5px;">
    <h3 id="italic">
        2.1 Renaming the time coloumn
    </h3>
</div>

In [6]:
data = data.rename(columns={"time": "date"})

<div style="font-family:verdana; word-spacing:1.5px;">
    <h3 id="italic">
        2.2 Converting it to the appropriate date format
    </h3>
</div>

In [7]:
data['date'] = pd.to_datetime(data['date'])

<div style="font-family:verdana; word-spacing:1.5px;">
    <h3 id="italic">
        2.3 Indexing the date and droping the column
    </h3>
</div>

In [8]:
data.set_index('date',drop=True, inplace=True)

<div style="font-family:verdana; word-spacing:1.5px;">
    <h3 id="italic">
        2.4 Resampling on a monthly bases
    </h3>
</div>

In [9]:
data = data.resample('M').mean()

In [10]:
data.head()

,visibility,windBearing,temperature,dewPoint,pressure,apparentTemperature,windSpeed,humidity
date,,,,,,,,
2011-11-30,8.215986,162.320833,10.595708,8.669917,1017.059139,10.004958,3.530500,0.883556
2011-12-31,12.519261,243.370968,7.087527,4.142728,1011.889664,4.624368,5.076720,0.819892
2012-01-31,11.484866,215.395161,6.776707,3.705578,1021.070349,4.701922,4.373333,0.813750
2012-02-29,10.541695,210.794540,4.868534,1.270101,1029.737687,2.829899,3.442011,0.783851
2012-03-31,9.320403,191.595430,9.307433,4.746989,1027.735833,8.151828,2.933952,0.752702


<h1 id="basics" style="font-family:verdana;"> 
    <center>3. Visualizing by animation</center>
</h1>

<div style="font-family:verdana; word-spacing:1.5px;">
    <h3 id="italic">
        3.1 The average temperature of the city
    </h3>
</div>

In [11]:
#Defining the start and end dates
END = data.index[-1]
START = data.index[0]


@gif.frame
def plot(df, date):
    df = df.loc[df.index[0]:pd.Timestamp(date)]
    fig, (ax1) = plt.subplots(1, figsize=(10, 5), dpi=100)
    ax1.plot(df.temperature, marker='o', linestyle='--', linewidth=3, markersize=10, color='tab:pink')
    maxi = round(df.temperature.max() + 5)
    ax1.set_title('Average Monthly Temperature in London over the 2012-2014', fontsize=20)
    ax1.set_xlim([START, END])
    ax1.set_ylim([0, maxi])
    ax1.set_ylabel('Temperature C°', color='tab:blue', fontsize=17)

In [12]:
frames = []
for date in pd.date_range(start=data.index[0],
                          end=data.index[-1], 
                          freq='1M'):
    frame = plot(data, date)
    frames.append(frame)

In [13]:
#saving the GIF
gif.save(frames, "output.gif", duration=0.5 ,unit = 's')

#visualizing the GIF
HTML('<img src="./output.gif" />')

<div style="font-family:verdana; word-spacing:1.5px;">
    <h3 id="italic">
        3.2 Adding some colour sections into the Animation
    </h3>
</div>

In [14]:
@gif.frame
def plot_split(df, date, split_date):
    df = df.loc[df.index[0]:pd.Timestamp(date)]
    fig, (ax1) = plt.subplots(1, figsize=(10, 5), dpi=100)

    #TRAIN
    if date < pd.Timestamp(split_date):
        ax1.axvspan(START, date, alpha=0.5, color='#33FF92')
        ax1.text(pd.Timestamp('2012-01-31'), y=12, s='Train', fontsize=20)  # where the train text goes
    #TEST
    if (date > pd.Timestamp(split_date)):
        ax1.axvspan(pd.Timestamp(split_date), date, alpha=0.5, color='#F933FF')
        ax1.text(pd.Timestamp('2014-01-31'), y=12, s='Test', fontsize=20)
    ax1.plot(df.temperature, marker='o', linestyle='--', linewidth=3, markersize=10, color='tab:orange')
    maxi = round(df.temperature.max() + 5)

    ax1.set_title('Train/Test-Split', fontsize=20)
    ax1.set_xlim([START, END])
    ax1.set_ylim([0, maxi])
    ax1.set_ylabel('Temperature C°', color='tab:blue', fontsize=17)

In [15]:
frames = []
for date in pd.date_range(start=data.index[0], end=data.index[-1], freq='1M'):
    frame = plot_split(data, date, '2013-06-21')
    frames.append(frame)

gif.save(frames, "output2.gif", duration=0.5, unit='s')

In [16]:
HTML('<img src="./output2.gif" />')

<div style="font-family:verdana; word-spacing:1.5px;">
    <h3 id="italic">
        3.3 Subplot Animation
    </h3>
</div>

In [17]:
@gif.frame
def plot(df, date):
    df = df.loc[df.index[0]:pd.Timestamp(date)]
    fig, (ax1, ax2, ax3) = plt.subplots(3, figsize=(10, 5), dpi=100)

    ax1.plot(df.temperature, marker='o', linestyle='--', linewidth=2, markersize=8, color='g')
    maxi = round(df.temperature.max() + 5)
    ax1.set_title('SUBPLOTS', fontsize=20)
    ax1.set_xlim([START, END])
    ax1.set_ylim([0, maxi])
    ax1.set_ylabel('Temperature C°', color='green', fontsize=10)

    ax2.plot(df.visibility, marker='o', linestyle='--', linewidth=2, markersize=8, color='r')
    maxi = round(df.visibility.max() + 3)
    ax2.set_xlim([START, END])
    ax2.set_ylim([0, maxi])
    ax2.set_ylabel('Visibility', color='red', fontsize=10)

    ax3.plot(df.windSpeed, marker='o', linestyle='--', linewidth=2, markersize=8, color='b')
    maxi = round(df.windSpeed.max() + 3)
    ax3.set_xlim([START, END])
    ax3.set_ylim([0, maxi])
    ax3.set_ylabel('Windspeed', color='blue', fontsize=10)


##### CREATE ANIMATIONS ######
frames = []
for date in pd.date_range(start=data.index[0], end=data.index[-1], freq='1M'):
    frame = plot(data, date)
    frames.append(frame)
gif.save(frames, "output3.gif", duration=0.5, unit='s')

In [18]:
HTML('<img src="./output3.gif" />')